# Thinking 1 ： 在CTR点击率预估中，使用GBDT+LR的原理是什么？

答：  
一、模型结构及需要去解决的问题：  
CTR 预估本质上是一个分类问题，分类问题可以想到的最基本的解决方法是 LR 模型。  
但 CTR 预估模型中存在以下问题：
1. 使用的数据集大；
2. LR 是通常使用的模型，但 LR 虽然能够快速的学习，但不能做到精准的学习非线性特征；
3. 人工的特征提取和构造成本太高，需要大量的经验。

模型结构：  
在以上的背景下，我们为了能够解决 LR 模型特征的选取问题，采用 GBDT 的树模型，先构造特征，然后使用 GBDT 树模型的叶子节点特征进行 LR 的分析。 stacking 思想 
串行的结构： GBDT -->> LR

二、GBDT 的原理：  
GBDT 树模型采用 boosting 的思想。  
GBDT 构造 k 个树，采用 CART 方法分裂，进行回归树建立，每个树学习上一颗树的残差，是一个总体上的串行结构。

三、LR 模型中的特征选择：正则化项  
由于自动提取的特征可能过多（树的叶子节点），使用正则化项惩罚过多的特征。

# Thinking 2 ：Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）

答：  
Wide & Deep 模型分为两个部分，一部分为 Wide（线性部分）：强调记忆性；一部分为 Deep （非线性神经网络部分）：强调模型的泛化能力。  
1. Wide 线性回归部分，直接使用原始的特征以及原始特征的交叉项，只要原始特征给出，就可以给它在模型中体现出来，强调记忆性，构造的特征和选择的特征均为人工提取的特征。
2. Deep 非线性神经网络部分， 这部分使用的特征需要先进行 embedding，然后使用神经网络进行非线性的学习，其学习的是非线性部分，得到的是前面没出现过的特征，因此强调其泛化能力。

# Thinking 3 ：在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？

答：  
Deep FM ；NFM 
1. Deep FM 使用并行的方式，相当于将Wide& Deep中的 LR 替换为了 FM。
2. NFM 采用串行的方式，在使用 FM 继续宁二阶特征提取后，再进行神经网络的拟合。

# Thinking 4 ：GBDT和随机森林都是基于树的算法，它们有什么区别？

答：  
GBDT 和 随机森林 都是集成学习的方法，但有些许的不同：    
1. GBDT 是 Boosting 思想，采用串行的方式，将多个分类器集成在一起，每一个弱分类器是对上一个分类器的残差进行分析。  
2. RF 随机森林是 Bagging 思想，采用并行的方式将多个弱分类器集成在一起，通过投票的方式，采用多数的结果为最后结果。

# Thinking 5 ： item流行度在推荐系统中有怎样的应用

答：  
1. 商品流行度解决冷启动问题：当没有用户太多的行为时，使用热门商品进行推荐。
2. 改进用户之间相似度问题：流行商品不好体现用户的个性化特征，因此使用基于流行度的权重对两个用户之间的相似度进行加权。
3. 改进商品的推荐次序：流行商品由于不好体现用户个性化特征，因此再推荐时，对商品基于流行度进行降权，较少推荐流行商品。

# Action 1 ： 使用Wide&Deep模型对movielens进行评分预测

In [5]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import WDL, NFM
from deepctr.feature_column import SparseFeat,get_feature_names

data = pd.read_csv("movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
    
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

Train on 128 samples, validate on 32 samples
128/128 [==============================] - 1s 5ms/sample - loss: 14.6953 - mean_squared_error: 14.6953 - val_loss: 12.9068 - val_mean_squared_error: 12.9068
test RMSE 3.630592238189246


In [6]:
# 使用NFM进行训练
model2 = NFM(linear_feature_columns, dnn_feature_columns, task='regression')
model2.compile("adam", "mse", metrics=['mse'], )
history2 = model2.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用NFM进行预测
pred_ans2 = model2.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse2 = round(mean_squared_error(test[target].values, pred_ans2), 4)
rmse2 = mse2 ** 0.5
print("test RMSE", rmse2)

Train on 128 samples, validate on 32 samples
128/128 [==============================] - 1s 6ms/sample - loss: 14.6953 - mean_squared_error: 14.6953 - val_loss: 12.9123 - val_mean_squared_error: 12.9123
test RMSE 3.631294535010896


# 本章任务

In [3]:
import xlrd
data = xlrd.open_workbook('L5-2自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-7):
    print(table.row(row_index)[:2])
for row_index in range(nrows-6, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

25
[text:'原理', text:'GBDT+LR的原理']
[text:'原理', text:'Gradient Boosting原理']
[text:'原理', text:'使用GBDT进行新特征构造']
[text:'原理', text:'评价指标NE，Calibration，AUC']
[text:'原理', text:'TPRate，FPRate计算']
[text:'工具', text:'sklearn中的GBDT']
[text:'原理', text:'Wide & Deep模型']
[text:'原理', text:'Wide join Deep同时训练']
[text:'工具', text:'DeepCTR中的WDL']
[text:'原理', text:'FNN模型']
[text:'原理', text:'NFM模型']
[text:'原理', text:'FM与DNN的两种结合方式：DeepFM，NFM']
[text:'工具', text:'DeepCTR中的NFM']
[text:'原理', text:'基于流行度的推荐算法']
[text:'原理', text:'影响流行度的因素']
[text:'原理', text:'推荐系统的架构']
[text:'Thinking1', text:'在CTR点击率预估中，使用GBDT+LR的原理是什么？'] text:'能简要说明GBDT和LR在CTR预估中的作用（10point）'
[text:'Thinking2', text:'Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）'] text:'1、能简要说明Wide&Deep的模型（5point）\n2、如何具备记忆和泛化能力（5point）\n'
[text:'Thinking3', text:'在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？'] text:'1、能说出哪两种FM和DNN的组合方式（5points）\n2、能说出FM和DNN的组合出的算法（5points）'
[text:'Thinking4', text:'GBDT和随机森林都是基于树的算法，它们有什么区别？'] text:'能简要说明这两种基于树的